In [ ]:
from diffusers import EulerDiscreteScheduler
import numpy as np
import torch
import test_utils

scheduler = EulerDiscreteScheduler(
  num_train_timesteps = 1000,
  beta_start = 0.00085,
  beta_end = 0.012,
  beta_schedule = "scaled_linear",
  interpolation_type = "linear",
  prediction_type = "epsilon",
  steps_offset = 1,
  timestep_spacing = "leading",
  trained_betas = None,
  use_karras_sigmas = False
)

# print some properties for newly initialized schedulers
print(f"beta_schedule: {scheduler.beta_schedule}")
print(f"sigmas: {scheduler.sigmas}")
print(f"init_noise_sigma: {scheduler.init_noise_sigma}")
np.savetxt("..\com.doji.diffusers\Tests\Editor\Resources\euler_discrete_test_sigmas.txt", [scheduler.sigmas], fmt='%g', delimiter=',\n')

#print("betas:")
np.savetxt("..\com.doji.diffusers\Tests\Editor\Resources\euler_discrete_test_betas.txt", [scheduler.betas], fmt='%g', delimiter=',\n')
#print(formatted_string_with_newlines)

#print timesteps after calling set_timesteps(10)
print(f"\ntimesteps before: {scheduler.timesteps}")
scheduler.set_timesteps(10)
print(f"timesteps after: {scheduler.timesteps}")
print(f"init_noise_sigma: {scheduler.init_noise_sigma}")
np.savetxt("..\com.doji.diffusers\Tests\Editor\Resources\euler_discrete_test_sigmas_2.txt", [scheduler.sigmas], fmt='%g', delimiter=',\n')

sample = test_utils.scheduler_samples()
sample = np.array(sample).reshape((4, 3, 8, 8))
timestep = 901

'''
residual = torch.tensor(sample) * timestep / (timestep + 1)
prev_timestep = timestep - 1000 // 10
print(prev_timestep)
prev = scheduler._get_prev_sample(residual, timestep, prev_timestep, torch.tensor(sample))
np.savetxt("prev_901.txt", prev.flatten(), fmt='%f', delimiter=',', newline=',\n')

residual = torch.tensor(sample) * timestep / (timestep + 1)
prev_timestep = timestep - 1000 // 10
print(prev_timestep)
prev = scheduler._get_prev_sample(residual, timestep, prev_timestep, torch.tensor(sample))
np.savetxt("prev_801.txt", prev.flatten(), fmt='%f', delimiter=',', newline=',\n')
'''

print(sample)
print(scheduler.timesteps)
#sample = torch.rand((4, 3, 8, 8))
#np.savetxt("output.txt", sample.flatten(), fmt='%f', delimiter=',', newline=',\n')
#print(f"rand: {sample.flatten()}")
sample = torch.tensor(sample)
sample = sample * scheduler.init_noise_sigma

for i, t in enumerate(scheduler.timesteps):
  print(t)
  residual = torch.tensor(sample) * t / (t + 1)
  residual = scheduler.scale_model_input(residual, t)
  sample = scheduler.step(residual, t, sample).prev_sample
  #np.savetxt(f"..\com.doji.diffusers\Tests\Editor\Resources\euler_discrete_prev_{t}_{i}.txt", [sample.flatten()],  fmt='%g', delimiter=',\n')
print(f"result: {sample}")
np.savetxt("..\com.doji.diffusers\Tests\Editor\Resources\euler_discrete_test_expected_output.txt", [sample.flatten()], fmt='%g', delimiter=',\n')

#residual = 0.1 * sample
#dummy_past_residuals = [residual + 0.2, residual + 0.15, residual + 0.1, residual + 0.05]
#scheduler.ets = dummy_past_residuals[:]
#output = scheduler.step_prk(residual, 0, sample).prev_sample
#print(type(scheduler.cur_model_output))
#print(scheduler.cur_model_output)